In [8]:
# Cell 1
# -*- coding: utf-8 -*-
import os
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler, label_binarize
import numpy as np
import torch.backends.cudnn as cudnn
from datetime import datetime
import matplotlib.pyplot as plt  # NEW: for line charts

# Global settings / reproducibility
cudnn.benchmark = True
torch.cuda.empty_cache()
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# Root path to UCRArchive_2018
ROOT = r"D:\2025暑期科研\UCRArchive_2018\UCRArchive_2018"

In [15]:
# Cell 2
def clean_and_pad_timeseries(raw_2d, min_len=8, cap_len=None, pad_value=0.0,
                             per_sample_standardize=True, fixed_len=None):
    """
    Clean time series with tail NaNs, z-score per-sample (optional), and pad/clip.

    Priority of output length:
      1) fixed_len: if not None, output length = fixed_len (force)
      2) cap_len:   if not None, output length = min(max_real_len, cap_len)
      3) otherwise, output length = max_real_len of this input batch
    """
    import numpy as np

    N, T = raw_2d.shape
    rows, keep_idx, real_lens = [], [], []

    for i in range(N):
        row = raw_2d[i]
        valid_vals = row[~np.isnan(row)]
        L = valid_vals.shape[0]
        if L < min_len:
            continue
        if per_sample_standardize:
            mu = valid_vals.mean()
            sigma = valid_vals.std()
            valid_vals = (valid_vals - mu) / (sigma if sigma > 0 else 1.0)
        rows.append(valid_vals); keep_idx.append(i); real_lens.append(L)

    if len(rows) == 0:
        raise ValueError("All samples filtered out. Lower min_len if needed.")

    max_real_len = max(real_lens)
    if fixed_len is not None:
        target_len = int(fixed_len)
    elif cap_len is not None:
        target_len = min(max_real_len, cap_len)
    else:
        target_len = max_real_len

    out = []
    for arr in rows:
        if arr.shape[0] >= target_len:
            arr = arr[:target_len]
        else:
            arr = np.pad(arr, (0, target_len - arr.shape[0]), constant_values=pad_value)
        out.append(arr)

    X = np.stack(out, axis=0).astype("float32")
    return X, np.array(keep_idx, dtype=np.int64), np.array(real_lens, dtype=np.int64)

In [16]:
# Cell 3
class TwoTowerTransformer(nn.Module):
    """
    Two-tower Transformer:
      - Tower 1: time series tokens [B, T, 1] -> embed -> transformer
      - Tower 2: Aout vector [B, F] as a single token -> embed -> transformer
      - Concat tokens -> final transformer -> flatten -> FC for multi-class logits
    """
    def __init__(self, input_dim1, input_dim2,
                 hidden_dim1, hidden_dim2, hidden_dim3,
                 num_heads, num_layers, num_classes,
                 seq_len1, seq_len2):
        super().__init__()
        # To keep it simple we force equal hidden dims and divisibility by nhead
        assert hidden_dim1 == hidden_dim2 == hidden_dim3, "hidden dims must be equal in this version."
        for h in (hidden_dim1, hidden_dim2, hidden_dim3):
            assert h % num_heads == 0, "hidden_dim must be divisible by num_heads"

        self.embedding1 = nn.Linear(input_dim1, hidden_dim1)
        self.embedding2 = nn.Linear(input_dim2, hidden_dim2)
        self.relu = nn.ReLU()

        self.transformer1 = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_dim1, nhead=num_heads, batch_first=True),
            num_layers=num_layers)

        self.transformer2 = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_dim2, nhead=num_heads, batch_first=True),
            num_layers=num_layers)

        self.final_transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_dim3, nhead=num_heads, batch_first=True),
            num_layers=num_layers)

        self.dropout = nn.Dropout(0.3)
        self.seq_len1 = seq_len1
        self.seq_len2 = 1  # treat Aout as a single token
        fc_in = hidden_dim3 * (seq_len1 + self.seq_len2)
        self.fc = nn.Linear(fc_in, num_classes)

    def forward(self, x1, x2):
        # x1: [B, T, input_dim1], x2: [B, F] or [B, 1, F]
        if x1.dim() == 2:
            x1 = x1.unsqueeze(-1)
        x1 = self.relu(self.embedding1(x1))
        x1 = self.transformer1(x1)

        if x2.dim() == 3:
            assert x2.size(1) == 1, "Expect x2 with L2=1 if 3D"
            x2 = x2.squeeze(1)
        x2 = self.relu(self.embedding2(x2))
        x2 = x2.unsqueeze(1)
        x2 = self.transformer2(x2)

        x = torch.cat((x1, x2), dim=1)
        x = self.final_transformer(x)
        x = x.reshape(x.size(0), -1)
        x = self.dropout(x)
        return self.fc(x)


class VisitDataset(Dataset):
    """Simple tensor dataset for (visit time series, aout features, one-hot labels)."""
    def __init__(self, visit_x, aout_x, y):
        self.visit_x = visit_x.astype("float32")
        self.aout_x  = aout_x.astype("float32")
        self.y       = y.astype("float32")
    def __len__(self): return len(self.y)
    def __getitem__(self, idx):
        return self.visit_x[idx], self.aout_x[idx], self.y[idx]

In [17]:
# Cell 4
def run_one_dataset(dataset_dir, dataset_name,
                    device='cuda:0',
                    batch_size=8, num_epochs=100, lr=1e-4,
                    cap_len=None,              # kept for compat; fixed_len dominates
                    patience=15,               # early stopping patience
                    verbose=False, plot_curves=False,
                    return_epoch_losses=False,   # ← 若为 True，则返回 test_loss_hist
                    force_full_epochs=False,     # True 时无视早停，跑满 num_epochs
                    collect_test_curve=False,    # ← 若为 True，逐 epoch 记录 TEST loss
                    return_epoch_curves=False):  # ← 若为 True，返回 (train_curve, test_curve)
    import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
    import torch, torch.nn as nn, torch.optim as optim
    from torch.utils.data import DataLoader
    from torch.optim.lr_scheduler import CosineAnnealingLR
    from sklearn.preprocessing import StandardScaler, label_binarize
    from sklearn.metrics import accuracy_score, roc_auc_score

    # ---------- paths ----------
    tsv_train_path = os.path.join(dataset_dir, f"{dataset_name}_TRAIN_cleaned.tsv")
    tsv_test_path  = os.path.join(dataset_dir, f"{dataset_name}_TEST_cleaned.tsv")
    aout_train_csv = os.path.join(dataset_dir, f"{dataset_name}_Aout_train_k2.csv")
    aout_test_csv  = os.path.join(dataset_dir, f"{dataset_name}_Aout_test_k2.csv")
    if not all(os.path.exists(p) for p in [tsv_train_path, tsv_test_path, aout_train_csv, aout_test_csv]):
        print(f"⚠ Skip {dataset_name}, missing files"); return None

    # ---------- read (OFFICIAL split; do NOT concat) ----------
    tsv_tr = pd.read_csv(tsv_train_path, sep="\t", header=None)
    tsv_te = pd.read_csv(tsv_test_path,  sep="\t", header=None)
    csv_tr = pd.read_csv(aout_train_csv, header=None)
    csv_te = pd.read_csv(aout_test_csv,  header=None)

    y_tr_raw = tsv_tr.iloc[:,0].values
    y_te_raw = tsv_te.iloc[:,0].values
    visit_tr_raw = tsv_tr.iloc[:,1:].values.astype("float32")
    visit_te_raw = tsv_te.iloc[:,1:].values.astype("float32")
    aout_tr_raw_all  = csv_tr.iloc[:,1:].values.astype("float32")
    aout_te_raw_all  = csv_te.iloc[:,1:].values.astype("float32")

    # ---------- (1) determine TRAIN reference length ----------
    tmp_train_clean, keep_tr0, _ = clean_and_pad_timeseries(
        visit_tr_raw, min_len=8, cap_len=None, pad_value=0.0,
        per_sample_standardize=True, fixed_len=None
    )
    train_seq_len = tmp_train_clean.shape[1]

    # ---------- (2) re-clean TRAIN/TEST with fixed_len=train_seq_len ----------
    visit_tr_clean, keep_tr, _ = clean_and_pad_timeseries(
        visit_tr_raw, min_len=8, cap_len=None, pad_value=0.0,
        per_sample_standardize=True, fixed_len=train_seq_len
    )
    y_tr = y_tr_raw[keep_tr]
    aout_tr_raw = aout_tr_raw_all[keep_tr]

    visit_te_clean, keep_te, _ = clean_and_pad_timeseries(
        visit_te_raw, min_len=8, cap_len=None, pad_value=0.0,
        per_sample_standardize=True, fixed_len=train_seq_len
    )
    y_te = y_te_raw[keep_te]
    aout_te_raw = aout_te_raw_all[keep_te]

    # ---------- (3) scaler: fit on TRAIN only ----------
    scaler = StandardScaler().fit(aout_tr_raw)
    aout_tr = scaler.transform(aout_tr_raw).astype("float32")
    aout_te = scaler.transform(aout_te_raw).astype("float32")

    # ---------- (4) labels ----------
    classes = sorted(np.unique(y_tr))
    Y_tr = label_binarize(y_tr, classes=classes).astype("float32")
    Y_te = label_binarize(y_te, classes=classes).astype("float32")

    # left tower to [B, L, 1]
    visit_tr = visit_tr_clean[:, :, None]
    visit_te = visit_te_clean[:, :, None]

    # ---------- model dims ----------
    seq_len1 = visit_tr.shape[1]; seq_len2 = 1
    input_dim1 = visit_tr.shape[2]; input_dim2 = aout_tr.shape[1]
    num_classes = Y_tr.shape[1]

    hidden_dim1 = hidden_dim2 = hidden_dim3 = 16
    num_heads = 2; num_layers = 2

    device = torch.device(device if torch.cuda.is_available() else 'cpu')
    model = TwoTowerTransformer(
        input_dim1, input_dim2,
        hidden_dim1, hidden_dim2, hidden_dim3,
        num_heads, num_layers,
        num_classes,
        seq_len1, seq_len2
    ).to(device)
    if torch.cuda.device_count() > 1 and str(device).startswith('cuda'):
        model = nn.DataParallel(model, device_ids=[0,1])
    model = model.to(device)

    # loaders
    train_loader = DataLoader(VisitDataset(visit_tr, aout_tr, Y_tr),
                              batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=0)
    test_loader  = DataLoader(VisitDataset(visit_te, aout_te, Y_te),
                              batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=0)

    # loss/opt/sched
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = CosineAnnealingLR(optimizer, T_max=50, eta_min=0)

    # early stopping on TRAIN loss only
    class EarlyStopping:
        def __init__(self, patience=15, delta=0.0):
            self.patience = patience; self.delta = delta
            self.counter = 0; self.best = None; self.stop = False
        def step(self, train_loss):
            if self.best is None:
                self.best = train_loss
                return False
            if train_loss > self.best - self.delta:
                self.counter += 1
                if self.counter >= self.patience:
                    self.stop = True
            else:
                self.best = train_loss
                self.counter = 0
            return self.stop

    es = EarlyStopping(patience=patience)

    # logging
    log_dir = os.path.join(dataset_dir, "_twotower_logs"); os.makedirs(log_dir, exist_ok=True)
    log_file = open(os.path.join(log_dir, "log_twotower.txt"), "w", encoding="utf-8")
    train_loss_hist, test_loss_hist = [], []

    # ---------- train + (optional) per-epoch TEST loss ----------
    for epoch in range(num_epochs):
        # Train
        model.train()
        total_loss = 0.0
        for x1,x2,y in train_loader:
            x1 = x1.to(torch.float32).to(device)
            x2 = x2.to(torch.float32).to(device)
            y  = y.to(torch.float32).to(device)
            optimizer.zero_grad()
            o = model(x1,x2)
            l = criterion(o,y)
            l.backward(); optimizer.step()
            total_loss += l.item()

        scheduler.step()
        avg_train_loss = total_loss / max(1,len(train_loader))
        train_loss_hist.append(avg_train_loss)
        log_file.write(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {avg_train_loss:.4f}\n")

        # Per-epoch TEST loss (optional, 真·TEST loss)
        if collect_test_curve:
            model.eval()
            t_total = 0.0
            with torch.no_grad():
                for x1,x2,y in test_loader:
                    x1 = x1.to(torch.float32).to(device)
                    x2 = x2.to(torch.float32).to(device)
                    y  = y.to(torch.float32).to(device)
                    o = model(x1,x2)
                    l = criterion(o,y)
                    t_total += l.item()
            avg_test_loss = t_total / max(1, len(test_loader))
            test_loss_hist.append(avg_test_loss)

        # Early stop (only if not forcing full epochs)
        stopped = es.step(avg_train_loss)
        if stopped and not force_full_epochs:
            break

    log_file.close()

    # ---------- TEST evaluation (run once after training) ----------
    model.eval()
    tout=[]; tlab=[]
    with torch.no_grad():
        for x1,x2,y in test_loader:
            x1 = x1.to(torch.float32).to(device)
            x2 = x2.to(torch.float32).to(device)
            o = model(x1,x2)
            tout.append(o.detach().cpu().numpy()); tlab.append(y.numpy())
    tout = np.concatenate(tout); tlab = np.concatenate(tlab)
    try:
        test_auc = roc_auc_score(tlab, tout, multi_class='ovr')
    except:
        test_auc = roc_auc_score(tlab, tout)
    test_pred = np.argmax(tout, axis=1)
    test_true = np.argmax(tlab, axis=1)
    test_acc = accuracy_score(test_true, test_pred)
    test_n = len(tlab)

    print(f"[{dataset_name}] TEST AUC={test_auc:.4f}, TEST ACC={test_acc:.4f}, n_samples={test_n}")

    # ---------- returns ----------
    if return_epoch_curves:
        # 返回 (train_curve, test_curve)；若未采集 test，则 test_curve=None
        return test_auc, test_acc, test_n, train_loss_hist, (test_loss_hist if collect_test_curve else None)
    elif return_epoch_losses:
        # 只返回 TEST loss 曲线（若未采集，则为 None）
        return test_auc, test_acc, test_n, (test_loss_hist if collect_test_curve else None)
    else:
        return test_auc, test_acc, test_n


In [18]:
# Cell 5
def discover_datasets(root):
    """
    Discover dataset subfolders that contain all four required files:
      *_TRAIN_cleaned.tsv, *_TEST_cleaned.tsv, *_Aout_train_k2.csv, *_Aout_test_k2.csv
    Returns: a sorted list of dataset names (folder names).
    """
    names = []
    for name in sorted(os.listdir(root)):
        subdir = os.path.join(root, name)
        if not os.path.isdir(subdir):
            continue
        t_train = os.path.join(subdir, f"{name}_TRAIN_cleaned.tsv")
        t_test  = os.path.join(subdir, f"{name}_TEST_cleaned.tsv")
        a_train = os.path.join(subdir, f"{name}_Aout_train_k2.csv")
        a_test  = os.path.join(subdir, f"{name}_Aout_test_k2.csv")
        if all(os.path.exists(p) for p in [t_train, t_test, a_train, a_test]):
            names.append(name)
    return names

In [13]:
# Cell 6
def run_all_datasets(root, device='cuda:0',
                     batch_size=8, num_epochs=100, lr=1e-4,
                     cap_len=None, verbose=False,
                     patience=15):
    import os, pandas as pd, numpy as np
    from datetime import datetime

    dataset_names = discover_datasets(root)
    print(f"Found {len(dataset_names)} datasets:", dataset_names)

    rows = []
    for name in dataset_names:
        out = run_one_dataset(
            dataset_dir=os.path.join(root, name),
            dataset_name=name,
            device=device,
            batch_size=batch_size,
            num_epochs=num_epochs,
            lr=lr,
            cap_len=cap_len,
            patience=patience,     # 训练早停仅基于 Train Loss
            verbose=verbose,
            plot_curves=False,     # 不画图
            return_epoch_losses=False,  # 不需要返回每个 epoch 的 loss
            force_full_epochs=False     # 正式评估允许早停
        )
        if out is None:
            continue

        test_auc, test_acc, test_n = out
        rows.append((name, float(test_auc), float(test_acc), int(test_n)))

    if not rows:
        print("No dataset finished successfully.")
        return None

    # 仅包含这四列，匹配你示例的格式
    df = pd.DataFrame(rows, columns=["dataset", "test_auc", "test_acc", "n_samples"]).sort_values("dataset")

    # 统计（可留作打印参考）
    mean_auc = df["test_auc"].mean()
    mean_acc = df["test_acc"].mean()
    w_auc = (df["test_auc"] * df["n_samples"]).sum() / df["n_samples"].sum()
    w_acc = (df["test_acc"] * df["n_samples"]).sum() / df["n_samples"].sum()

    # 打印与示例一致的样式
    print("\n========== Summary (OFFICIAL TEST) ==========")
    print(df.to_string(index=False))
    print(f"\nSimple mean: AUC = {mean_auc:.4f}, ACC = {mean_acc:.4f}")
    print(f"Weighted (by samples): AUC = {w_auc:.4f}, ACC = {w_acc:.4f}")

    # 只保存 CSV（不写 txt）
    summary_dir = os.path.join(root, "_twotower_logs")
    os.makedirs(summary_dir, exist_ok=True)
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_csv = os.path.join(summary_dir, f"summary_TEST_{ts}.csv")
    df.to_csv(out_csv, index=False, encoding="utf-8")

    return df, (mean_auc, mean_acc), (w_auc, w_acc)


In [20]:
# Cell 7
# === BATCH COLLECTOR (TwoTower): build logs/twotower_testloss.tsv ===
import os, numpy as np, pandas as pd

NUM_EPOCHS = 60
LOG_DIR = "logs"
os.makedirs(LOG_DIR, exist_ok=True)

dataset_names = discover_datasets(ROOT)
print(f"[TwoTower] Discover {len(dataset_names)} datasets")

# 行：epoch 1..NUM_EPOCHS；列：各数据集
loss_table = pd.DataFrame(index=np.arange(1, NUM_EPOCHS+1),
                          columns=dataset_names, dtype=float)
loss_table.index.name = "epoch"

for ds in dataset_names:
    try:
        out = run_one_dataset(
            dataset_dir=os.path.join(ROOT, ds),
            dataset_name=ds,
            device='cuda:0',
            batch_size=8,
            num_epochs=NUM_EPOCHS,   # 固定 60
            lr=1e-4,
            cap_len=None,
            patience=15,
            verbose=False,
            plot_curves=False,
            force_full_epochs=True,        # 跑满 60，不早停
            collect_test_curve=True,       # ← 开启逐 epoch 计算 TEST loss
            return_epoch_losses=True       # ← 返回 test_loss_hist（不是 train）
        )
        if out is None:
            print(f"[SKIP] {ds} returned None")
            loss_table[ds] = np.nan
            continue

        # 期望结构：out = (test_auc, test_acc, test_n, test_loss_hist)
        if len(out) != 4:
            print(f"[WARN] {ds}: unexpected return length={len(out)}")
        _, _, _, test_curve = out

        if test_curve is None:
            print(f"[WARN] {ds}: test_curve is None (collect_test_curve=False?)")
            loss_table[ds] = np.nan
            continue

        # 对齐到固定行数（多了裁，少了补 NaN）
        epoch_losses = (list(test_curve) + [np.nan]*NUM_EPOCHS)[:NUM_EPOCHS]
        loss_table[ds] = epoch_losses

    except Exception as e:
        print(f"[ERROR] {ds}: {e}")
        loss_table[ds] = np.nan

out_path = os.path.join(LOG_DIR, "twotower_testloss.tsv")
loss_table.to_csv(out_path, sep="\t", float_format="%.6f")
print(f"[TwoTower] Saved test-loss table to {out_path} with shape {loss_table.shape}")



[TwoTower] Discover 125 datasets
[ACSF1] TEST AUC=0.9458, TEST ACC=0.7500, n_samples=100
[Adiac] TEST AUC=0.9603, TEST ACC=0.5064, n_samples=391
[AllGestureWiimoteX] TEST AUC=0.7795, TEST ACC=0.3730, n_samples=681
[AllGestureWiimoteY] TEST AUC=0.8644, TEST ACC=0.4837, n_samples=645
[AllGestureWiimoteZ] TEST AUC=0.7964, TEST ACC=0.3328, n_samples=685
[ArrowHead] TEST AUC=0.8602, TEST ACC=0.6114, n_samples=175
[BME] TEST AUC=0.9353, TEST ACC=0.6800, n_samples=150
[Beef] TEST AUC=0.8875, TEST ACC=0.7000, n_samples=30
[BeetleFly] TEST AUC=0.8100, TEST ACC=1.0000, n_samples=20
[BirdChicken] TEST AUC=0.6700, TEST ACC=1.0000, n_samples=20
[CBF] TEST AUC=0.9920, TEST ACC=0.9589, n_samples=900
[Car] TEST AUC=0.8641, TEST ACC=0.6833, n_samples=60
[Chinatown] TEST AUC=0.9911, TEST ACC=1.0000, n_samples=343
[ChlorineConcentration] TEST AUC=0.6571, TEST ACC=0.5729, n_samples=3840
[CinCECGTorso] TEST AUC=0.9941, TEST ACC=0.9196, n_samples=1380
[Coffee] TEST AUC=0.9846, TEST ACC=1.0000, n_samples=28


In [8]:
# Cell 8
# Quiet run with per-dataset one-line summaries + curves + early stopping:
result = run_all_datasets(
    root=ROOT,
    device='cuda:0',
    batch_size=8,
    num_epochs=100,
    lr=1e-4,
    cap_len=None,
    verbose=False,
    patience=15,        # 训练早停仅基于 Train Loss
)


Found 125 datasets: ['ACSF1', 'Adiac', 'AllGestureWiimoteX', 'AllGestureWiimoteY', 'AllGestureWiimoteZ', 'ArrowHead', 'BME', 'Beef', 'BeetleFly', 'BirdChicken', 'CBF', 'Car', 'Chinatown', 'ChlorineConcentration', 'CinCECGTorso', 'Coffee', 'CricketX', 'CricketY', 'CricketZ', 'Crop', 'DiatomSizeReduction', 'DistalPhalanxOutlineAgeGroup', 'DistalPhalanxOutlineCorrect', 'DistalPhalanxTW', 'DodgerLoopDay', 'DodgerLoopGame', 'DodgerLoopWeekend', 'ECG200', 'ECG5000', 'ECGFiveDays', 'EOGHorizontalSignal', 'EOGVerticalSignal', 'Earthquakes', 'EthanolLevel', 'FaceAll', 'FaceFour', 'FacesUCR', 'FiftyWords', 'Fish', 'FordA', 'FordB', 'FreezerRegularTrain', 'FreezerSmallTrain', 'Fungi', 'GestureMidAirD1', 'GestureMidAirD2', 'GestureMidAirD3', 'GesturePebbleZ1', 'GesturePebbleZ2', 'GunPoint', 'GunPointAgeSpan', 'GunPointMaleVersusFemale', 'GunPointOldVersusYoung', 'Ham', 'HandOutlines', 'Haptics', 'Herring', 'HouseTwenty', 'InlineSkate', 'InsectEPGRegularTrain', 'InsectEPGSmallTrain', 'InsectWingbea

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
